In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:05<00:00, 31.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
X_train = torch.tensor(train_data.data, dtype=torch.float32)
y_train = torch.tensor(train_data.targets,dtype = torch.float32)

X_test = torch.tensor(test_data.data, dtype=torch.float32)
y_test = torch.tensor(test_data.targets,dtype = torch.float32)

In [ ]:
X_train = X_train.permute(0,3,1,2)
X_test = X_test.permute(0,3,1,2)

In [ ]:
class CNNDataset(Dataset):
  def __init__(self,X_train,y_train):
    self.X_train = X_train
    self.y_train = y_train

  def __len__(self):

    return len(self.y_train)

  def __getitem__(self, idx):

    return X_train[idx],y_train[idx]

In [ ]:
dataset = CNNDataset(X_train,y_train)

dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
class CNNModel(nn.Module):
  def __init__(self, num_classes):
    super(CNNModel, self).__init__()

    self.func1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(kernel_size=2, stride=2),
                           nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(kernel_size=2, stride=2),
                           nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                           nn.ReLU(),
                           nn.MaxPool2d(kernel_size=2, stride=2)
                           )

    self.func2 = nn.Sequential(nn.Linear(128*4*4, 256),
                           nn.ReLU(),
                           nn.Dropout(0.4),
                           nn.Linear(256, num_classes)
                           )


  def forward(self, x):
    x = self.func1(x)
    x = x.view(x.size(0), -1)
    x = self.func2(x)
    return x

In [ ]:
model = CNNModel(num_classes=len(y_train.unique())).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(50):
    model.train()
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels.long())


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 1.5535
Epoch [2/10], Loss: 1.5045
Epoch [3/10], Loss: 1.2674
Epoch [4/10], Loss: 1.4734
Epoch [5/10], Loss: 1.1095
Epoch [6/10], Loss: 0.9212
Epoch [7/10], Loss: 0.8832
Epoch [8/10], Loss: 0.8261
Epoch [9/10], Loss: 0.7830
Epoch [10/10], Loss: 0.7521
Epoch [11/10], Loss: 0.6269
Epoch [12/10], Loss: 0.8518
Epoch [13/10], Loss: 0.6911
Epoch [14/10], Loss: 0.7316
Epoch [15/10], Loss: 0.6667
Epoch [16/10], Loss: 0.5045
Epoch [17/10], Loss: 0.5671
Epoch [18/10], Loss: 0.4317
Epoch [19/10], Loss: 0.3804
Epoch [20/10], Loss: 0.6322
Epoch [21/10], Loss: 0.5232
Epoch [22/10], Loss: 0.2654
Epoch [23/10], Loss: 0.2485
Epoch [24/10], Loss: 0.4628
Epoch [25/10], Loss: 0.3671
Epoch [26/10], Loss: 0.3432
Epoch [27/10], Loss: 0.3076
Epoch [28/10], Loss: 0.1706
Epoch [29/10], Loss: 0.2227
Epoch [30/10], Loss: 0.2462
Epoch [31/10], Loss: 0.2923
Epoch [32/10], Loss: 0.1304
Epoch [33/10], Loss: 0.2276
Epoch [34/10], Loss: 0.1863
Epoch [35/10], Loss: 0.2944
Epoch [36/10], Loss: 0.1955
E

In [ ]:
correct = 0
total = 0
test_loader = DataLoader(dataset=CNNDataset(X_test,y_test), batch_size=100, shuffle=False)
model.eval()

with torch.no_grad():
    for images, labels in test_loader:

        images, labels = images.to(device), labels.to(device)


        outputs = model(images)


        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on the test dataset: {accuracy:.2f}%")

Accuracy on the test dataset: 99.55%
